In [1]:
from dotenv import load_dotenv
import os
import requests
import json
import pandas as pd
import datetime as dt
from pathlib import Path

import numpy as np
# %matplotlib inline
# import matplotlib
# import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [3]:
load_dotenv() 

True

In [4]:
api_key = os.getenv("ZIPCODE_API_KEY")
zip_code = 77002
distance = 5
units = "miles"
url = f"http://www.zipcodeapi.com/rest/{api_key}/radius.json/{zip_code}/{distance}/{units}"

In [5]:
response_data = requests.get(url)
data = response_data.json()
print(json.dumps(data, indent=4))

{
    "zip_codes": [
        {
            "zip_code": "77021",
            "distance": 4.071,
            "city": "Houston",
            "state": "TX"
        },
        {
            "zip_code": "77030",
            "distance": 4.082,
            "city": "Houston",
            "state": "TX"
        },
        {
            "zip_code": "77005",
            "distance": 4.396,
            "city": "Houston",
            "state": "TX"
        },
        {
            "zip_code": "77023",
            "distance": 3.714,
            "city": "Houston",
            "state": "TX"
        },
        {
            "zip_code": "77004",
            "distance": 2.211,
            "city": "Houston",
            "state": "TX"
        },
        {
            "zip_code": "77046",
            "distance": 4.39,
            "city": "Houston",
            "state": "TX"
        },
        {
            "zip_code": "77098",
            "distance": 3.418,
            "city": "Houston",
            "state": "T

In [6]:
zip_code = []
city = []

for line in data["zip_codes"]:

    new_zip_code = int(line["zip_code"])
    zip_code.append(new_zip_code)
    
    new_city = line["city"]
    city.append(new_city)
    
zip_code_df = pd.DataFrame()
zip_code_df['zip_code']  = zip_code
zip_code_df['City']  = city

def filter_zip_code(zip_codes_list, df, new_dataframe):
    for zip_code_numbers in zip_codes_list:
        if zip_code_numbers in df:
            data_adding = df.loc[:, zip_code_numbers]
            new_dataframe = new_dataframe.append(data_adding)
    return new_dataframe   




In [ ]:
# importing the data
clean_path = Path("../resources/Vu/Tax_Data/cleaned_tax_FINALs.csv")

clean_tax = pd.read_csv(clean_path)

clean_tax = clean_tax.rename({'Zip Code': 'zip_code'}, axis=1)

clean_tax = clean_tax.astype({'zip_code': 'int'})

clean_tax.drop(columns = "Size of adjusted gross income", inplace = True)

multi_index_tax = clean_tax.groupby(["zip_code", "date"]).sum()


# creating a new dataframe for each of the columns
number_of_returns_df = pd.pivot_table(multi_index_tax, index = "date", values = "Number of returns", columns = "zip_code")

taxable_income_df = pd.pivot_table(multi_index_tax, index = "date", values = "Taxable income", columns = "zip_code")

income_tax_df = pd.pivot_table(multi_index_tax, index = "date", values = "Income tax", columns = "zip_code")

mortgage_tax_df = pd.pivot_table(multi_index_tax, index = "date", values = "Mortgage", columns = "zip_code")

charity_tax_df = pd.pivot_table(multi_index_tax, index = "date", values = "Charity", columns = "zip_code")

real_estates_tax_df = pd.pivot_table(multi_index_tax, index = "date", values = "Real Estates", columns = "zip_code")



# Setting index to datetime and sorting index(date)
number_of_returns_df.index = pd.to_datetime(number_of_returns_df.index)
number_of_returns_df.sort_index(inplace=True)

taxable_income_df.index = pd.to_datetime(taxable_income_df.index)
taxable_income_df.sort_index(inplace=True)

income_tax_df.index = pd.to_datetime(income_tax_df.index)
income_tax_df.sort_index(inplace=True)

mortgage_tax_df.index = pd.to_datetime(mortgage_tax_df.index)
mortgage_tax_df.sort_index(inplace=True)

charity_tax_df.index = pd.to_datetime(charity_tax_df.index)
charity_tax_df.sort_index(inplace=True)

real_estates_tax_df.index = pd.to_datetime(real_estates_tax_df.index)
real_estates_tax_df.sort_index(inplace=True)

In [8]:
permits_filepath = Path("../resources/dan/Permit_Data/permit_data.csv")

permits_df = pd.read_csv(permits_filepath)

permits_df.dropna(inplace=True)

permits_df = permits_df.astype({'zip_code': 'int64'})

permits_df.set_index(pd.to_datetime(permits_df['date'], format='%Y%m%d', infer_datetime_format=True), inplace=True)

permits_df.drop('date', axis=1, inplace=True)

permit_counts = permits_df.groupby(["zip_code", "date"]).sum()

permits_by_zip = pd.pivot_table(permit_counts, index = "date", values = "count", columns = "zip_code")

# permits_by_zip.index = permits_by_zip.index.to_period('M')

permits_by_zip = permits_by_zip.groupby(permits_by_zip.index).sum()

permits_by_zip.fillna(0, inplace=True)

permits_by_zip = permits_by_zip.astype(int)

In [9]:
csvpath = Path("../resources/Jack/Texas_Pricing_Data.csv")

pricing_data_df = pd.read_csv(csvpath)

pricing_data_df.set_index(pd.to_datetime(pricing_data_df['month_date_yyyymm'], format='%Y%m', infer_datetime_format=True), inplace=True)

pricing_data_df.index.rename('date', inplace=True)

pricing_data_df.index.rename('date', inplace=True)

pricing_data_df.sort_values(by=['postal_code','date'], inplace=True)

pricing_data_df['pct_price_change'] = 0

pricing_data_df['pct_price_sqft_change'] = 0

pricing_data_df['pct_price_sqft_change'] = pricing_data_df['median_listing_price_per_square_feet'].pct_change()

pricing_data_df.index.rename('date', inplace=True)

median_list_sqft_df = pd.pivot_table(pricing_data_df, index='date', columns = 'postal_code', values = 'median_listing_price_per_square_feet')

median_list_sqft_df_pct_change = median_list_sqft_df.pct_change()

active_listings_df = pd.pivot_table(pricing_data_df, index='date', columns = 'postal_code', values = 'active_listing_count')

new_listing_df = pd.pivot_table(pricing_data_df, index='date', columns = 'postal_code', values = 'new_listing_count')

average_listing_price_df = pd.pivot_table(pricing_data_df, index='date', columns = 'postal_code', values = 'average_listing_price')

pending_ratio_df = pd.pivot_table(pricing_data_df, index='date', columns = 'postal_code', values = 'pending_ratio')

In [10]:
pricing_data_df.tail()

,month_date_yyyymm,postal_code,zip_name,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,pending_listing_count,median_listing_price_per_square_feet,median_square_feet,average_listing_price,total_listing_count,pending_ratio,pct_price_change,pct_price_sqft_change
date,,,,,,,,,,,,,,,,,
2020-02-01,202002,77995,"yoakum, tx",209850.0,29,133.0,8,0,4,9,100.147275,2103.0,342624,38,0.3103,0,0.033128
2020-03-01,202003,77995,"yoakum, tx",202400.0,29,142.0,0,0,0,11,91.252346,2050.0,326075,40,0.3793,0,-0.088818
2020-04-01,202004,77995,"yoakum, tx",194500.0,31,105.0,8,0,0,4,99.498926,1917.0,295784,35,0.1290,0,0.090371
2020-05-01,202005,77995,"yoakum, tx",198550.0,31,120.0,4,0,8,8,100.147275,2103.0,328709,39,0.2581,0,0.006516
2020-06-01,202006,77995,"yoakum, tx",219550.0,27,99.5,4,0,0,12,98.611774,2103.5,359262,39,0.4444,0,-0.015332


In [11]:
filtered_median_list_sqft_df = pd.DataFrame()
filtered_median_list_sqft_df = filter_zip_code(zip_code, median_list_sqft_df, filtered_median_list_sqft_df)
filtered_median_list_sqft_df.fillna(0, inplace = True)
filtered_median_list_sqft_df = filtered_median_list_sqft_df.transpose()
filtered_median_list_sqft_df.reset_index(inplace = True)


filtered_median_list_sqft_df_pct_change = pd.DataFrame()
filtered_median_list_sqft_df_pct_change = filter_zip_code(zip_code, median_list_sqft_df_pct_change, filtered_median_list_sqft_df_pct_change)
filtered_median_list_sqft_df_pct_change.fillna(0, inplace = True)
filtered_median_list_sqft_df_pct_change = filtered_median_list_sqft_df_pct_change.transpose()
filtered_median_list_sqft_df_pct_change.reset_index(inplace = True)


filtered_active_listings_df = pd.DataFrame()
filtered_active_listings_df = filter_zip_code(zip_code, active_listings_df, filtered_active_listings_df)
filtered_active_listings_df.fillna(0, inplace = True)
filtered_active_listings_df = filtered_active_listings_df.transpose()
filtered_active_listings_df.reset_index(inplace = True)



filtered_new_listing_df = pd.DataFrame()
filtered_new_listing_df = filter_zip_code(zip_code, new_listing_df, filtered_new_listing_df)
filtered_new_listing_df.fillna(0, inplace = True)
filtered_new_listing_df = filtered_new_listing_df.transpose()
filtered_new_listing_df.reset_index(inplace = True)



filtered_average_listing_price_df = pd.DataFrame()
filtered_average_listing_price_df = filter_zip_code(zip_code, average_listing_price_df, filtered_average_listing_price_df)
filtered_average_listing_price_df.fillna(0, inplace = True)
filtered_average_listing_price_df = filtered_average_listing_price_df.transpose()
filtered_average_listing_price_df.reset_index(inplace = True)



filtered_pending_ratio_df = pd.DataFrame()
filtered_pending_ratio_df = filter_zip_code(zip_code, pending_ratio_df, filtered_pending_ratio_df)
filtered_pending_ratio_df.fillna(0, inplace = True)
filtered_pending_ratio_df = filtered_pending_ratio_df.transpose()
filtered_pending_ratio_df.reset_index(inplace = True)


In [ ]:
d.transpose()

In [12]:
filtered_permits_by_zip = pd.DataFrame()
filtered_permits_by_zip = filter_zip_code(zip_code, permits_by_zip, filtered_permits_by_zip)
filtered_permits_by_zip.fillna(0, inplace = True)
filtered_permits_by_zip = filtered_permits_by_zip.transpose()
filtered_permits_by_zip.reset_index(inplace = True)


In [13]:
filtered_average_listing_price_df[77007]

0     580182.0
1     572637.0
2     573682.0
3     568499.0
4     571030.0
5     565998.0
6     576562.0
7     572364.0
8     574004.0
9     565722.0
10    571387.0
11    575573.0
12    572544.0
13    575222.0
14    577751.0
15    580528.0
16    575912.0
17    583825.0
18    571782.0
19    581770.0
20    570541.0
21    588097.0
22    602706.0
23    601284.0
24    588195.0
25    581449.0
26    576634.0
27    576308.0
28    591316.0
29    595021.0
30    602078.0
31    622265.0
32    624617.0
33    621292.0
34    605750.0
35    613176.0
36    606757.0
37    605109.0
38    602438.0
39    591788.0
40    599788.0
41    606090.0
42    615808.0
43    634083.0
44    641174.0
45    657397.0
46    672104.0
47    667553.0
Name: 77007, dtype: float64

In [14]:
for i in zip_code:
    if i in filtered_median_list_sqft_df and i in filtered_median_list_sqft_df_pct_change and i in filtered_active_listings_df and i in filtered_new_listing_df and i in filtered_average_listing_price_df and i in filtered_pending_ratio_df and i in filtered_permits_by_zip and i in filtered_number_of_returns_df and i in filtered_taxable_income_df and i in filtered_income_tax_df and i in filtered_mortgage_tax_df and i in filtered_charity_tax_df and i in filtered_real_estates_tax_df:
        dict_of_frames[i] = pd.DataFrame({
            'Median Listing Sqft' : filtered_median_list_sqft_df[i],
            'Median Listing of Sqft Pct Change' : filtered_median_list_sqft_df_pct_change[i], 
            'Active Listings' : filtered_active_listings_df[i], 
            'New Listings' : filtered_new_listing_df[i], 
            'Average Listing Price' : filtered_average_listing_price_df[i],
            'Pending Ratio' : filtered_pending_ratio_df[i],
            'Number of Permits' : filtered_permits_by_zip[i],
            'Number of Tax Returns' : filtered_number_of_returns_df[i],
            'Average Taxable Income' : filtered_taxable_income_df[i],
            'Average Income Tax' : filtered_income_tax_df[i],
            'Average Mortgage Tax' : filtered_mortgage_tax_df[i],
            'Average Charity Tax' : filtered_charity_tax_df[i],
            'Average Real Estates Tax' : filtered_real_estates_tax_df[i]
            })
        dict_of_frames[i].dropna(inplace = True)

NameError: name 'dict_of_frames' is not defined

In [141]:
dict_of_frames[77021]

,Median Listing Sqft,Median Listing of Sqft Pct Change,Active Listings,New Listings,Average Listing Price,Pending Ratio,Number of Permits,Number of Tax Returns,Average Taxable Income,Average Income Tax,Average Mortgage Tax,Average Charity Tax,Average Real Estates Tax
0,142.178911,0.000000,68.0,24.0,317253.0,0.4412,1.0,11926.0,240880.0,38948.0,9820.0,9328.0,5422.0
1,139.786769,-0.016825,67.0,24.0,325472.0,0.3731,3.0,11926.0,240880.0,38948.0,9820.0,9328.0,5422.0
2,139.377302,-0.002929,71.0,32.0,323803.0,0.3521,2.0,11926.0,240880.0,38948.0,9820.0,9328.0,5422.0
3,139.664805,0.002063,74.0,28.0,343343.0,0.3784,1.0,11926.0,240880.0,38948.0,9820.0,9328.0,5422.0
4,144.994568,0.038161,73.0,12.0,351859.0,0.4110,0.0,11926.0,240880.0,38948.0,9820.0,9328.0,5422.0
5,141.120278,-0.026720,64.0,12.0,351338.0,0.4375,0.0,11926.0,240880.0,38948.0,9820.0,9328.0,5422.0
6,136.942675,-0.029603,69.0,32.0,346408.0,0.3333,1.0,11926.0,240880.0,38948.0,9820.0,9328.0,5422.0
7,138.595819,0.012072,62.0,12.0,351963.0,0.4516,0.0,11926.0,240880.0,38948.0,9820.0,9328.0,5422.0
8,140.706704,0.015231,62.0,28.0,361069.0,0.4355,0.0,11926.0,240880.0,38948.0,9820.0,9328.0,5422.0
9,136.022514,-0.033290,74.0,32.0,351055.0,0.3919,0.0,11926.0,240880.0,38948.0,9820.0,9328.0,5422.0


In [15]:

dict_of_corr = {}

for i in zip_code:
    if i in filtered_median_list_sqft_df and i in filtered_median_list_sqft_df_pct_change and i in filtered_active_listings_df and i in filtered_new_listing_df and i in filtered_average_listing_price_df and i in filtered_pending_ratio_df and i in filtered_permits_by_zip and i in filtered_number_of_returns_df and i in filtered_taxable_income_df and i in filtered_income_tax_df and i in filtered_mortgage_tax_df and i in filtered_charity_tax_df and i in filtered_real_estates_tax_df:
        dict_of_corr[i] = pd.DataFrame({
            'Median Listing Sqft' : filtered_median_list_sqft_df[i],
            'Median Listing of Sqft Pct Change' : filtered_median_list_sqft_df_pct_change[i], 
            'Active Listings' : filtered_active_listings_df[i], 
            'New Listings' : filtered_new_listing_df[i], 
            'Average Listing Price' : filtered_average_listing_price_df[i],
            'Pending Ratio' : filtered_pending_ratio_df[i],
            'Number of Permits' : filtered_permits_by_zip[i],
            'Number of Tax Returns' : filtered_number_of_returns_df[i],
            'Average Taxable Income' : filtered_taxable_income_df[i],
            'Average Income Tax' : filtered_income_tax_df[i],
            'Average Mortgage Tax' : filtered_mortgage_tax_df[i],
            'Average Charity Tax' : filtered_charity_tax_df[i],
            'Average Real Estates Tax' : filtered_real_estates_tax_df[i]
            }).corr()
        np.fill_diagonal(dict_of_corr[i].values, -2)


In [16]:
dict_of_corr.keys()

dict_keys([77021, 77030, 77005, 77023, 77004, 77046, 77098, 77027, 77006, 77011, 77003, 77019, 77002, 77007, 77020, 77009, 77008, 77026])

In [150]:
dict_of_corr

{77021:                                    Median Listing Sqft  \
 Median Listing Sqft                          -2.000000   
 Median Listing of Sqft Pct Change             0.426909   
 Active Listings                               0.575154   
 New Listings                                  0.075691   
 Average Listing Price                        -0.192950   
 Pending Ratio                                -0.277074   
 Number of Permits                             0.011637   
 Number of Tax Returns                         0.633219   
 Average Taxable Income                        0.716547   
 Average Income Tax                            0.703750   
 Average Mortgage Tax                         -0.741157   
 Average Charity Tax                           0.729239   
 Average Real Estates Tax                      0.432113   
 
                                    Median Listing of Sqft Pct Change  \
 Median Listing Sqft                                         0.426909   
 Median Listing of 

In [119]:
# filtered_median_list_sqft_df[77021]
filtered_median_list_sqft_df_pct_change.reset_index(inplace = True)
# filtered_median_list_sqft_df_pct_change.drop["index"]
# filtered_active_listings_df[i], 
# filtered_new_listing_df[i], 
# filtered_average_listing_price_df[i],
# filtered_pending_ratio_df[i],
# filtered_permits_by_zip[77021],
# filtered_number_of_returns_df[i],
# filtered_taxable_income_df[i],
# filtered_income_tax_df[i],
# filtered_mortgage_tax_df[i],
# filtered_charity_tax_df[i],
# filtered_real_estates_tax_df

ValueError: cannot insert level_0, already exists

In [17]:
# dict_of_frames = {}
# lr = LinearRegression()

# for i in dict_of_corr.keys():
#     if i in filtered_median_list_sqft_df and i in filtered_median_list_sqft_df_pct_change and i in filtered_active_listings_df and i in filtered_new_listing_df and i in filtered_average_listing_price_df and i in filtered_pending_ratio_df and i in filtered_permits_by_zip and i in filtered_number_of_returns_df and i in filtered_taxable_income_df and i in filtered_income_tax_df and i in filtered_mortgage_tax_df and i in filtered_charity_tax_df and i in filtered_real_estates_tax_df:
#         dict_of_frames[i] = pd.DataFrame({
#             'Median Listing Sqft' : filtered_median_list_sqft_df[i],
#             'Median Listing of Sqft Pct Change' : filtered_median_list_sqft_df_pct_change[i], 
#             'Active Listings' : filtered_active_listings_df[i], 
#             'New Listings' : filtered_new_listing_df[i], 
#             'Average Listing Price' : filtered_average_listing_price_df[i],
#             'Pending Ratio' : filtered_pending_ratio_df[i],
#             'Number of Permits' : filtered_permits_by_zip[i],
#             'Number of Tax Returns' : filtered_number_of_returns_df[i],
#             'Average Taxable Income' : filtered_taxable_income_df[i],
#             'Average Income Tax' : filtered_income_tax_df[i],
#             'Average Mortgage Tax' : filtered_mortgage_tax_df[i],
#             'Average Charity Tax' : filtered_charity_tax_df[i],
#             'Average Real Estates Tax' : filtered_real_estates_tax_df[i]
#             })
#     dict_of_frames[i].dropna(inplace=True)
#     max_df = dict_of_corr[i].agg({'Median Listing of Sqft Pct Change' : ['max']})
#     if max_df.values[0][0] == -2:
#         print(f"Data Set Incomplete: Removing {[i]}")
#         #del dict_of_frames[i]
#     else:
#         strong_corr = dict_of_corr[i].index[dict_of_corr[i]['Median Listing of Sqft Pct Change'] == max_df.values[0][0]].tolist()
#         print(f"{strong_corr[0]} {[i]}")
#         lr.fit(dict_of_frames[i][[strong_corr[0]]], dict_of_frames[i]['Median Listing of Sqft Pct Change'])


dict_of_frames = {}
lr = LinearRegression()


for i in dict_of_corr.keys():
    if i in filtered_median_list_sqft_df and i in filtered_median_list_sqft_df_pct_change and i in filtered_active_listings_df and i in filtered_new_listing_df and i in filtered_average_listing_price_df and i in filtered_pending_ratio_df and i in filtered_permits_by_zip and i in filtered_number_of_returns_df and i in filtered_taxable_income_df and i in filtered_income_tax_df and i in filtered_mortgage_tax_df and i in filtered_charity_tax_df and i in filtered_real_estates_tax_df:
        dict_of_frames[i] = pd.DataFrame({
#             'Median Listing Sqft' : filtered_median_list_sqft_df[i],
            'Median Listing of Sqft Pct Change' : filtered_median_list_sqft_df_pct_change[i], 
            'Active Listings' : filtered_active_listings_df[i], 
            'New Listings' : filtered_new_listing_df[i], 
            'Average Listing Price' : filtered_average_listing_price_df[i],
            'Pending Ratio' : filtered_pending_ratio_df[i],
            'Number of Permits' : filtered_permits_by_zip[i],
            'Number of Tax Returns' : filtered_number_of_returns_df[i],
            'Average Taxable Income' : filtered_taxable_income_df[i],
            'Average Income Tax' : filtered_income_tax_df[i],
            'Average Mortgage Tax' : filtered_mortgage_tax_df[i],
            'Average Charity Tax' : filtered_charity_tax_df[i],
            'Average Real Estates Tax' : filtered_real_estates_tax_df[i]
            })
    dict_of_frames[i].dropna(inplace=True)
    max_df = dict_of_corr[i].agg({'Median Listing of Sqft Pct Change' : ['max']})
    if max_df.values[0][0] == -2:
        print(f"Data Set Incomplete: Removing {[i]}")
        del dict_of_frames[i]
    else:
        strong_corr = dict_of_corr[i].index[dict_of_corr[i]['Median Listing of Sqft Pct Change'] == max_df.values[0][0]].tolist()
        print(f"{strong_corr[0]} {[i]}")
        index_df = pd.DataFrame(np.array(dict_of_frames[i].index))
        lr.fit(index_df[[0]], dict_of_frames[i][[strong_corr[0]]])
        future = lr.predict([index_df.iloc[-1]+12])
        lr.fit(dict_of_frames[i][[strong_corr[0]]], dict_of_frames[i]['Median Listing of Sqft Pct Change'])
        print(f"ROI:") 
        print(lr.predict(future))

Median Listing Sqft [77021]


KeyError: "None of [Index(['Median Listing Sqft'], dtype='object')] are in the [columns]"

In [144]:
dict_of_frames[77003]

,Median Listing Sqft,Median Listing of Sqft Pct Change,Active Listings,New Listings,Average Listing Price,Pending Ratio,Number of Permits,Number of Tax Returns,Average Taxable Income,Average Income Tax,Average Mortgage Tax,Average Charity Tax,Average Real Estates Tax
0,183.107833,0.000000,98.0,40.0,341451.0,0.5102,0.0,5226.0,152194.0,25651.0,12329.0,2163.0,4447.0
1,183.223494,0.000632,99.0,40.0,334126.0,0.4646,0.0,5226.0,152194.0,25651.0,12329.0,2163.0,4447.0
2,181.604937,-0.008834,98.0,40.0,340273.0,0.4490,0.0,5226.0,152194.0,25651.0,12329.0,2163.0,4447.0
3,176.812555,-0.026389,98.0,32.0,342834.0,0.4082,2.0,5226.0,152194.0,25651.0,12329.0,2163.0,4447.0
4,185.854849,0.051141,116.0,48.0,359987.0,0.4310,2.0,5226.0,152194.0,25651.0,12329.0,2163.0,4447.0
5,192.133387,0.033782,109.0,12.0,360752.0,0.3578,1.0,5226.0,152194.0,25651.0,12329.0,2163.0,4447.0
6,201.467895,0.048583,113.0,88.0,373879.0,0.4248,0.0,5226.0,152194.0,25651.0,12329.0,2163.0,4447.0
7,206.659677,0.025770,122.0,32.0,376707.0,0.4344,1.0,5226.0,152194.0,25651.0,12329.0,2163.0,4447.0
8,204.806805,-0.008966,117.0,32.0,376533.0,0.4188,2.0,5226.0,152194.0,25651.0,12329.0,2163.0,4447.0
9,209.527170,0.023048,126.0,36.0,376382.0,0.4206,1.0,5226.0,152194.0,25651.0,12329.0,2163.0,4447.0


In [126]:
filtered_new_listing_df

,index,77021,77030,77005,77023,77004,77046,77098,77027,77006,77011,77003,77019,77002,77007,77020,77009,77008,77026
0,2016-07-01,24.0,24.0,56.0,20.0,68.0,0.0,24.0,20.0,60.0,8.0,40.0,64.0,12.0,200.0,20.0,80.0,128.0,8.0
1,2016-08-01,24.0,20.0,40.0,4.0,72.0,0.0,28.0,36.0,48.0,0.0,40.0,60.0,12.0,156.0,20.0,68.0,128.0,16.0
2,2016-09-01,32.0,24.0,52.0,20.0,64.0,0.0,32.0,32.0,68.0,4.0,40.0,64.0,12.0,144.0,8.0,72.0,120.0,4.0
3,2016-10-01,28.0,20.0,48.0,24.0,56.0,0.0,28.0,32.0,60.0,4.0,32.0,56.0,8.0,156.0,8.0,80.0,156.0,8.0
4,2016-11-01,12.0,4.0,24.0,8.0,64.0,0.0,20.0,32.0,68.0,4.0,48.0,56.0,16.0,156.0,4.0,68.0,96.0,4.0
5,2016-12-01,12.0,12.0,12.0,4.0,56.0,0.0,8.0,12.0,20.0,8.0,12.0,32.0,8.0,64.0,8.0,32.0,60.0,4.0
6,2017-01-01,32.0,20.0,48.0,24.0,100.0,0.0,40.0,52.0,68.0,4.0,88.0,64.0,16.0,184.0,32.0,88.0,208.0,8.0
7,2017-02-01,12.0,28.0,52.0,20.0,56.0,0.0,32.0,48.0,64.0,12.0,32.0,72.0,12.0,168.0,4.0,84.0,144.0,8.0
8,2017-03-01,28.0,24.0,56.0,16.0,76.0,0.0,44.0,52.0,76.0,8.0,32.0,76.0,16.0,192.0,20.0,96.0,152.0,8.0
9,2017-04-01,32.0,24.0,60.0,24.0,64.0,0.0,24.0,52.0,60.0,4.0,36.0,88.0,16.0,208.0,20.0,88.0,164.0,8.0


z
c


In [105]:
filtered_median_list_sqft_df = pd.DataFrame()
filtered_median_list_sqft_df = filter_zip_code(zip_code, median_list_sqft_df, filtered_median_list_sqft_df)
filtered_median_list_sqft_df.fillna(0, inplace = True)

In [110]:
type(median_list_sqft_df)

pandas.core.frame.DataFrame

In [111]:
median_list_sqft_df.rest_index()

AttributeError: 'DataFrame' object has no attribute 'rest_index'

In [230]:
old = pd.DataFrame()
old = filter_zip_code(zip_code, permits_by_zip, old)

In [231]:
type(old)

pandas.core.frame.DataFrame

In [232]:
old

,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,...,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02
77021,4.0,3.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,3.0,8.0,5.0,3.0,2.0,6.0,1.0,10.0,4.0,4.0
77030,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0
77005,0.0,0.0,1.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0
77023,8.0,3.0,0.0,2.0,0.0,3.0,1.0,0.0,0.0,1.0,...,0.0,0.0,2.0,2.0,3.0,1.0,1.0,0.0,0.0,1.0
77004,11.0,9.0,5.0,1.0,6.0,2.0,6.0,5.0,2.0,2.0,...,2.0,7.0,4.0,2.0,6.0,17.0,4.0,6.0,3.0,13.0
77046,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77098,1.0,1.0,1.0,0.0,0.0,2.0,2.0,0.0,3.0,2.0,...,1.0,0.0,0.0,2.0,3.0,4.0,1.0,0.0,1.0,4.0
77027,2.0,3.0,3.0,1.0,2.0,0.0,1.0,1.0,1.0,1.0,...,2.0,3.0,2.0,5.0,2.0,4.0,0.0,2.0,0.0,3.0
77006,4.0,1.0,2.0,2.0,4.0,2.0,2.0,4.0,3.0,1.0,...,7.0,4.0,0.0,1.0,2.0,6.0,0.0,4.0,2.0,6.0
77011,0.0,0.0,2.0,0.0,2.0,0.0,1.0,0.0,2.0,2.0,...,5.0,2.0,2.0,2.0,1.0,0.0,1.0,0.0,0.0,3.0


In [225]:
new = pd.DataFrame()

In [226]:
type(new)

pandas.core.frame.DataFrame

In [227]:
old = filter_zip_code(zip_code, permits_by_zip, new)

In [228]:
old

<bound method DataFrame.transpose of        2017-01  2017-02  2017-03  2017-04  2017-05  2017-06  2017-07  2017-08  \
77021      4.0      3.0      0.0      1.0      0.0      0.0      1.0      0.0   

       2017-09  2017-10  ...  2019-05  2019-06  2019-07  2019-08  2019-09  \
77021      0.0      0.0  ...      3.0      8.0      5.0      3.0      2.0   

       2019-10  2019-11  2019-12  2020-01  2020-02  
77021      6.0      1.0     10.0      4.0      4.0  

[1 rows x 38 columns]>

In [310]:
csvpath = Path("../resources/Jack/Texas_Pricing_Data.csv")

pricing_data_df = pd.read_csv(csvpath)

pricing_data_df.set_index(pd.to_datetime(pricing_data_df['month_date_yyyymm'], format='%Y%m', infer_datetime_format=True), inplace=True)

pricing_data_df.index.rename('date', inplace=True)

pricing_data_df.index.rename('date', inplace=True)

pricing_data_df.sort_values(by=['postal_code','date'], inplace=True)

pricing_data_df['pct_price_change'] = 0

pricing_data_df['pct_price_sqft_change'] = 0

pricing_data_df['pct_price_sqft_change'] = pricing_data_df['median_listing_price_per_square_feet'].pct_change()

pricing_data_df.index.rename('date', inplace=True)

median_list_sqft_df = pd.pivot_table(pricing_data_df, index='date', columns = 'postal_code', values = 'median_listing_price_per_square_feet')

median_list_sqft_df_pct_change = median_list_sqft_df.pct_change()

active_listings_df = pd.pivot_table(pricing_data_df, index='date', columns = 'postal_code', values = 'active_listing_count')

new_listing_df = pd.pivot_table(pricing_data_df, index='date', columns = 'postal_code', values = 'new_listing_count')

average_listing_price_df = pd.pivot_table(pricing_data_df, index='date', columns = 'postal_code', values = 'average_listing_price')

pending_ratio_df = pd.pivot_table(pricing_data_df, index='date', columns = 'postal_code', values = 'pending_ratio')



In [239]:
average_new = pd.DataFrame()
average_new = filter_zip_code(zip_code, median_list_sqft_df, average_new)
average_new

,2016-07-01,2016-08-01,2016-09-01,2016-10-01,2016-11-01,2016-12-01,2017-01-01,2017-02-01,2017-03-01,2017-04-01,...,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01
77021,142.178911,139.786769,139.377302,139.664805,144.994568,141.120278,136.942675,138.595819,140.706704,136.022514,...,145.466796,148.197597,147.058824,146.842561,150.368929,149.389228,150.909091,151.410543,151.911996,154.650198
77030,278.160919,253.579952,244.391570,247.560535,233.305522,222.682119,213.800136,206.896552,224.999391,264.717907,...,230.296473,217.907228,214.752568,207.389428,208.736638,211.326712,211.713888,219.107389,221.238938,216.333625
77005,378.371983,369.056748,376.384644,379.631504,376.736656,368.528219,365.865463,368.241199,365.257757,367.492097,...,387.769037,388.208047,381.271676,381.856150,385.662032,387.438615,388.341979,391.148860,388.503194,391.775716
77023,171.188717,171.217815,171.270381,171.232877,170.175439,172.030025,179.471729,184.703110,170.258483,164.565826,...,192.307692,192.214034,191.568396,183.727381,182.750398,182.750398,188.304986,185.746166,190.771961,194.027992
77004,201.602896,197.478930,194.207222,193.965517,195.847630,194.188828,195.099895,191.908231,192.122712,198.982751,...,195.307503,189.816882,191.024166,191.024166,198.274292,199.483037,193.883291,192.363483,190.075321,192.477139
77046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,245.998272,243.664717,245.379222,245.379222,240.085349,239.396118,229.547579,229.044834,224.574084,229.044834
77098,284.013605,283.770996,280.607419,282.738095,284.531816,285.289116,285.230109,285.230109,285.544083,283.880171,...,308.709838,320.150659,298.191612,313.932975,316.008040,296.355653,288.641582,294.235925,294.235925,297.268860
77027,296.749117,308.416034,301.685772,303.245436,304.678134,316.961524,303.183863,322.460126,327.713369,329.643093,...,352.162436,343.840877,363.427048,382.661172,388.605836,382.661172,378.137407,382.953653,390.220969,398.823783
77006,246.741936,251.914062,233.480908,235.253550,238.491771,250.116225,250.000000,252.401826,250.314465,243.750000,...,242.108349,243.909261,246.031746,246.754637,250.960529,250.169109,248.877253,243.626149,240.759593,240.481980
77011,115.740741,120.192308,123.259636,114.425872,108.877996,108.877996,114.425872,148.572042,114.199993,150.240385,...,166.406542,166.884566,162.679739,153.468816,143.884892,160.472973,160.472973,162.860890,152.329749,166.436377


In [241]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [279]:
zip_fit2 = np.polyfit(real_estates_tax_df.iloc[1], income_tax_df.iloc[1], 1)



In [291]:
real_estates_tax_df.index = pd.to_datetime(real_estates_tax_df.index)
real_estates_tax_df.sort_index(inplace=True)


In [292]:
real_estates_tax_df

zip_code,75001,75002,75006,75007,75009,75010,75013,75019,75020,75021,...,79925,79927,79928,79930,79932,79934,79935,79936,79938,99999
date,,,,,,,,,,,,,,,,,,,,,
2011-01-01,9619.0,66000.0,17647.0,36085.0,7922.0,19179.0,58280.0,67503.0,5403.0,1941.0,...,10476.0,3096.0,13964.0,2759.0,16379.0,8782.0,4988.0,34966.0,20203.0,273410.0
2011-02-01,9619.0,66000.0,17647.0,36085.0,7922.0,19179.0,58280.0,67503.0,5403.0,1941.0,...,10476.0,3096.0,13964.0,2759.0,16379.0,8782.0,4988.0,34966.0,20203.0,273410.0
2011-03-01,9619.0,66000.0,17647.0,36085.0,7922.0,19179.0,58280.0,67503.0,5403.0,1941.0,...,10476.0,3096.0,13964.0,2759.0,16379.0,8782.0,4988.0,34966.0,20203.0,273410.0
2011-04-01,9619.0,66000.0,17647.0,36085.0,7922.0,19179.0,58280.0,67503.0,5403.0,1941.0,...,10476.0,3096.0,13964.0,2759.0,16379.0,8782.0,4988.0,34966.0,20203.0,273410.0
2011-05-01,9619.0,66000.0,17647.0,36085.0,7922.0,19179.0,58280.0,67503.0,5403.0,1941.0,...,10476.0,3096.0,13964.0,2759.0,16379.0,8782.0,4988.0,34966.0,20203.0,273410.0
2011-06-01,9619.0,66000.0,17647.0,36085.0,7922.0,19179.0,58280.0,67503.0,5403.0,1941.0,...,10476.0,3096.0,13964.0,2759.0,16379.0,8782.0,4988.0,34966.0,20203.0,273410.0
2011-07-01,9619.0,66000.0,17647.0,36085.0,7922.0,19179.0,58280.0,67503.0,5403.0,1941.0,...,10476.0,3096.0,13964.0,2759.0,16379.0,8782.0,4988.0,34966.0,20203.0,273410.0
2011-08-01,9619.0,66000.0,17647.0,36085.0,7922.0,19179.0,58280.0,67503.0,5403.0,1941.0,...,10476.0,3096.0,13964.0,2759.0,16379.0,8782.0,4988.0,34966.0,20203.0,273410.0
2011-09-01,9619.0,66000.0,17647.0,36085.0,7922.0,19179.0,58280.0,67503.0,5403.0,1941.0,...,10476.0,3096.0,13964.0,2759.0,16379.0,8782.0,4988.0,34966.0,20203.0,273410.0


In [260]:
real_estates_tax_df.iloc[1].isnull().sum()

6

In [261]:
real_estates_tax_df.dropna(inplace= True)

In [277]:
income_tax_df.fillna(0, inplace = True)

In [278]:
real_estates_tax_df.fillna(0, inplace = True)

In [281]:
filtered_income_tax_df = pd.DataFrame()
filtered_income_tax_df = filter_zip_code(zip_code, income_tax_df, filtered_income_tax_df)

In [282]:
filtered_real_estates_tax_df = pd.DataFrame()
filtered_real_estates_tax_df = filter_zip_code(zip_code, real_estates_tax_df, filtered_real_estates_tax_df)

In [299]:
zip_fit3 = np.polyfit(filtered_real_estates_tax_df.iloc[1], filtered_permits_by_zip.iloc[1], 1)


TypeError: expected x and y to have same length

In [324]:
zip_fit3

array([ 1.11526588e+01, -3.03265804e+04])

In [320]:
filtered_real_estates_tax_df.iloc[1]

IndexError: single positional indexer is out-of-bounds

In [321]:
filtered_permits_by_zip = pd.DataFrame()
filtered_permits_by_zip = filter_zip_code(zip_code, permits_by_zip, filtered_permits_by_zip)

In [322]:
filtered_permits_by_zip

,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,...,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02
77021,4.0,3.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,3.0,8.0,5.0,3.0,2.0,6.0,1.0,10.0,4.0,4.0
77030,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0
77005,0.0,0.0,1.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0
77023,8.0,3.0,0.0,2.0,0.0,3.0,1.0,0.0,0.0,1.0,...,0.0,0.0,2.0,2.0,3.0,1.0,1.0,0.0,0.0,1.0
77004,11.0,9.0,5.0,1.0,6.0,2.0,6.0,5.0,2.0,2.0,...,2.0,7.0,4.0,2.0,6.0,17.0,4.0,6.0,3.0,13.0
77046,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77098,1.0,1.0,1.0,0.0,0.0,2.0,2.0,0.0,3.0,2.0,...,1.0,0.0,0.0,2.0,3.0,4.0,1.0,0.0,1.0,4.0
77027,2.0,3.0,3.0,1.0,2.0,0.0,1.0,1.0,1.0,1.0,...,2.0,3.0,2.0,5.0,2.0,4.0,0.0,2.0,0.0,3.0
77006,4.0,1.0,2.0,2.0,4.0,2.0,2.0,4.0,3.0,1.0,...,7.0,4.0,0.0,1.0,2.0,6.0,0.0,4.0,2.0,6.0
77011,0.0,0.0,2.0,0.0,2.0,0.0,1.0,0.0,2.0,2.0,...,5.0,2.0,2.0,2.0,1.0,0.0,1.0,0.0,0.0,3.0


In [343]:
filtered_number_of_returns_df = pd.DataFrame()
filtered_number_of_returns_df = filter_zip_code(zip_code, number_of_returns_df, filtered_number_of_returns_df)

filtered_taxable_income_df = pd.DataFrame()
filtered_taxable_income_df = filter_zip_code(zip_code, taxable_income_df, filtered_taxable_income_df)

filtered_income_tax_df = pd.DataFrame()
filtered_income_tax_df = filter_zip_code(zip_code, income_tax_df, filtered_income_tax_df)

filtered_mortgage_tax_df = pd.DataFrame()
filtered_mortgage_tax_df = filter_zip_code(zip_code, mortgage_tax_df, filtered_mortgage_tax_df)

filtered_charity_tax_df = pd.DataFrame()
filtered_charity_tax_df = filter_zip_code(zip_code, charity_tax_df, filtered_charity_tax_df)

filtered_real_estates_tax_df = pd.DataFrame()
filtered_real_estates_tax_df = filter_zip_code(zip_code, real_estates_tax_df, filtered_real_estates_tax_df)



In [344]:
filtered_charity_tax_df

,2011-01-01,2011-02-01,2011-03-01,2011-04-01,2011-05-01,2011-06-01,2011-07-01,2011-08-01,2011-09-01,2011-10-01,...,2015-03-01,2015-04-01,2015-05-01,2015-06-01,2015-07-01,2015-08-01,2015-09-01,2015-10-01,2015-11-01,2015-12-01
77021,9328.0,9328.0,9328.0,9328.0,9328.0,9328.0,9328.0,9328.0,9328.0,9328.0,...,13302.0,13302.0,13302.0,13302.0,13302.0,13302.0,13302.0,13302.0,13302.0,13302.0
77030,12360.0,12360.0,12360.0,12360.0,12360.0,12360.0,12360.0,12360.0,12360.0,12360.0,...,15973.0,15973.0,15973.0,15973.0,15973.0,15973.0,15973.0,15973.0,15973.0,15973.0
77005,87794.0,87794.0,87794.0,87794.0,87794.0,87794.0,87794.0,87794.0,87794.0,87794.0,...,123103.0,123103.0,123103.0,123103.0,123103.0,123103.0,123103.0,123103.0,123103.0,123103.0
77023,2887.0,2887.0,2887.0,2887.0,2887.0,2887.0,2887.0,2887.0,2887.0,2887.0,...,4094.0,4094.0,4094.0,4094.0,4094.0,4094.0,4094.0,4094.0,4094.0,4094.0
77004,17314.0,17314.0,17314.0,17314.0,17314.0,17314.0,17314.0,17314.0,17314.0,17314.0,...,22436.0,22436.0,22436.0,22436.0,22436.0,22436.0,22436.0,22436.0,22436.0,22436.0
77046,3786.0,3786.0,3786.0,3786.0,3786.0,3786.0,3786.0,3786.0,3786.0,3786.0,...,4651.0,4651.0,4651.0,4651.0,4651.0,4651.0,4651.0,4651.0,4651.0,4651.0
77098,26453.0,26453.0,26453.0,26453.0,26453.0,26453.0,26453.0,26453.0,26453.0,26453.0,...,33747.0,33747.0,33747.0,33747.0,33747.0,33747.0,33747.0,33747.0,33747.0,33747.0
77027,47038.0,47038.0,47038.0,47038.0,47038.0,47038.0,47038.0,47038.0,47038.0,47038.0,...,63421.0,63421.0,63421.0,63421.0,63421.0,63421.0,63421.0,63421.0,63421.0,63421.0
77006,21580.0,21580.0,21580.0,21580.0,21580.0,21580.0,21580.0,21580.0,21580.0,21580.0,...,25854.0,25854.0,25854.0,25854.0,25854.0,25854.0,25854.0,25854.0,25854.0,25854.0
77011,679.0,679.0,679.0,679.0,679.0,679.0,679.0,679.0,679.0,679.0,...,1399.0,1399.0,1399.0,1399.0,1399.0,1399.0,1399.0,1399.0,1399.0,1399.0


In [312]:
filtered_median_list_sqft_df = pd.DataFrame()
filtered_median_list_sqft_df = filter_zip_code(zip_code, median_list_sqft_df, filtered_median_list_sqft_df)

filtered_median_list_sqft_df_pct_change = pd.DataFrame()
filtered_median_list_sqft_df_pct_change = filter_zip_code(zip_code, median_list_sqft_df_pct_change, filtered_median_list_sqft_df_pct_change)

filtered_active_listings_df = pd.DataFrame()
filtered_active_listings_df = filter_zip_code(zip_code, active_listings_df, filtered_active_listings_df)

filtered_new_listing_df = pd.DataFrame()
filtered_new_listing_df = filter_zip_code(zip_code, new_listing_df, filtered_new_listing_df)

filtered_average_listing_price_df = pd.DataFrame()
filtered_average_listing_price_df = filter_zip_code(zip_code, average_listing_price_df, filtered_average_listing_price_df)

filtered_pending_ratio_df = pd.DataFrame()
filtered_pending_ratio_df = filter_zip_code(zip_code, pending_ratio_df, filtered_pending_ratio_df)



In [318]:
filtered_median_list_sqft_df

,2016-07-01,2016-08-01,2016-09-01,2016-10-01,2016-11-01,2016-12-01,2017-01-01,2017-02-01,2017-03-01,2017-04-01,...,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01
77021,142.178911,139.786769,139.377302,139.664805,144.994568,141.120278,136.942675,138.595819,140.706704,136.022514,...,145.466796,148.197597,147.058824,146.842561,150.368929,149.389228,150.909091,151.410543,151.911996,154.650198
77030,278.160919,253.579952,244.391570,247.560535,233.305522,222.682119,213.800136,206.896552,224.999391,264.717907,...,230.296473,217.907228,214.752568,207.389428,208.736638,211.326712,211.713888,219.107389,221.238938,216.333625
77005,378.371983,369.056748,376.384644,379.631504,376.736656,368.528219,365.865463,368.241199,365.257757,367.492097,...,387.769037,388.208047,381.271676,381.856150,385.662032,387.438615,388.341979,391.148860,388.503194,391.775716
77023,171.188717,171.217815,171.270381,171.232877,170.175439,172.030025,179.471729,184.703110,170.258483,164.565826,...,192.307692,192.214034,191.568396,183.727381,182.750398,182.750398,188.304986,185.746166,190.771961,194.027992
77004,201.602896,197.478930,194.207222,193.965517,195.847630,194.188828,195.099895,191.908231,192.122712,198.982751,...,195.307503,189.816882,191.024166,191.024166,198.274292,199.483037,193.883291,192.363483,190.075321,192.477139
77046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,245.998272,243.664717,245.379222,245.379222,240.085349,239.396118,229.547579,229.044834,224.574084,229.044834
77098,284.013605,283.770996,280.607419,282.738095,284.531816,285.289116,285.230109,285.230109,285.544083,283.880171,...,308.709838,320.150659,298.191612,313.932975,316.008040,296.355653,288.641582,294.235925,294.235925,297.268860
77027,296.749117,308.416034,301.685772,303.245436,304.678134,316.961524,303.183863,322.460126,327.713369,329.643093,...,352.162436,343.840877,363.427048,382.661172,388.605836,382.661172,378.137407,382.953653,390.220969,398.823783
77006,246.741936,251.914062,233.480908,235.253550,238.491771,250.116225,250.000000,252.401826,250.314465,243.750000,...,242.108349,243.909261,246.031746,246.754637,250.960529,250.169109,248.877253,243.626149,240.759593,240.481980
77011,115.740741,120.192308,123.259636,114.425872,108.877996,108.877996,114.425872,148.572042,114.199993,150.240385,...,166.406542,166.884566,162.679739,153.468816,143.884892,160.472973,160.472973,162.860890,152.329749,166.436377


In [345]:
filtered_income_tax_df

,2011-01-01,2011-02-01,2011-03-01,2011-04-01,2011-05-01,2011-06-01,2011-07-01,2011-08-01,2011-09-01,2011-10-01,...,2015-03-01,2015-04-01,2015-05-01,2015-06-01,2015-07-01,2015-08-01,2015-09-01,2015-10-01,2015-11-01,2015-12-01
77021,38948.0,38948.0,38948.0,38948.0,38948.0,38948.0,38948.0,38948.0,38948.0,38948.0,...,53399.0,53399.0,53399.0,53399.0,53399.0,53399.0,53399.0,53399.0,53399.0,53399.0
77030,136049.0,136049.0,136049.0,136049.0,136049.0,136049.0,136049.0,136049.0,136049.0,136049.0,...,191894.0,191894.0,191894.0,191894.0,191894.0,191894.0,191894.0,191894.0,191894.0,191894.0
77005,902405.0,902405.0,902405.0,902405.0,902405.0,902405.0,902405.0,902405.0,902405.0,902405.0,...,1224135.0,1224135.0,1224135.0,1224135.0,1224135.0,1224135.0,1224135.0,1224135.0,1224135.0,1224135.0
77023,28406.0,28406.0,28406.0,28406.0,28406.0,28406.0,28406.0,28406.0,28406.0,28406.0,...,38288.0,38288.0,38288.0,38288.0,38288.0,38288.0,38288.0,38288.0,38288.0,38288.0
77004,125114.0,125114.0,125114.0,125114.0,125114.0,125114.0,125114.0,125114.0,125114.0,125114.0,...,159849.0,159849.0,159849.0,159849.0,159849.0,159849.0,159849.0,159849.0,159849.0,159849.0
77046,41883.0,41883.0,41883.0,41883.0,41883.0,41883.0,41883.0,41883.0,41883.0,41883.0,...,53257.0,53257.0,53257.0,53257.0,53257.0,53257.0,53257.0,53257.0,53257.0,53257.0
77098,301576.0,301576.0,301576.0,301576.0,301576.0,301576.0,301576.0,301576.0,301576.0,301576.0,...,331772.0,331772.0,331772.0,331772.0,331772.0,331772.0,331772.0,331772.0,331772.0,331772.0
77027,341339.0,341339.0,341339.0,341339.0,341339.0,341339.0,341339.0,341339.0,341339.0,341339.0,...,483210.0,483210.0,483210.0,483210.0,483210.0,483210.0,483210.0,483210.0,483210.0,483210.0
77006,251430.0,251430.0,251430.0,251430.0,251430.0,251430.0,251430.0,251430.0,251430.0,251430.0,...,340858.0,340858.0,340858.0,340858.0,340858.0,340858.0,340858.0,340858.0,340858.0,340858.0
77011,9803.0,9803.0,9803.0,9803.0,9803.0,9803.0,9803.0,9803.0,9803.0,9803.0,...,16256.0,16256.0,16256.0,16256.0,16256.0,16256.0,16256.0,16256.0,16256.0,16256.0


In [347]:
# mlr_zip
zip_fit3 = np.polyfit(filtered_real_estates_tax_df.iloc[1], filtered_income_tax_df.iloc[1], 1)


In [349]:
filtered_real_estates_tax_df.iloc[1]

2011-01-01    16705.0
2011-02-01    16705.0
2011-03-01    16705.0
2011-04-01    16705.0
2011-05-01    16705.0
2011-06-01    16705.0
2011-07-01    16705.0
2011-08-01    16705.0
2011-09-01    16705.0
2011-10-01    16705.0
2011-11-01    16705.0
2011-12-01    16705.0
2012-01-01    16867.0
2012-02-01    16867.0
2012-03-01    16867.0
2012-04-01    16867.0
2012-05-01    16867.0
2012-06-01    16867.0
2012-07-01    16867.0
2012-08-01    16867.0
2012-09-01    16867.0
2012-10-01    16867.0
2012-11-01    16867.0
2012-12-01    16867.0
2013-01-01    16679.0
2013-02-01    16679.0
2013-03-01    16679.0
2013-04-01    16679.0
2013-05-01    16679.0
2013-06-01    16679.0
2013-07-01    16679.0
2013-08-01    16679.0
2013-09-01    16679.0
2013-10-01    16679.0
2013-11-01    16679.0
2013-12-01    16679.0
2014-01-01    18278.0
2014-02-01    18278.0
2014-03-01    18278.0
2014-04-01    18278.0
2014-05-01    18278.0
2014-06-01    18278.0
2014-07-01    18278.0
2014-08-01    18278.0
2014-09-01    18278.0
2014-10-01

In [346]:
LinearRegression(filtered_real_estates_tax_df.iloc[1], filtered_income_tax_df.iloc[1])

C:\Users\dovu9\anaconda3\envs\dev\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass fit_intercept=2011-01-01    16705.0
2011-02-01    16705.0
2011-03-01    16705.0
2011-04-01    16705.0
2011-05-01    16705.0
2011-06-01    16705.0
2011-07-01    16705.0
2011-08-01    16705.0
2011-09-01    16705.0
2011-10-01    16705.0
2011-11-01    16705.0
2011-12-01    16705.0
2012-01-01    16867.0
2012-02-01    16867.0
2012-03-01    16867.0
2012-04-01    16867.0
2012-05-01    16867.0
2012-06-01    16867.0
2012-07-01    16867.0
2012-08-01    16867.0
2012-09-01    16867.0
2012-10-01    16867.0
2012-11-01    16867.0
2012-12-01    16867.0
2013-01-01    16679.0
2013-02-01    16679.0
2013-03-01    16679.0
2013-04-01    16679.0
2013-05-01    16679.0
2013-06-01    16679.0
2013-07-01    16679.0
2013-08-01    16679.0
2013-09-01    16679.0
2013-10-01    16679.0
2013-11-01    16679.0
2013-12-01    16679.0
2014-01-01    18278.0
2014-02-01    18278.0
2014-03-01    18278.0
2014-04-01    18278.0
20

LinearRegression(fit_intercept=2011-01-01    16705.0
2011-02-01    16705.0
2011-03-01    16705.0
2011-04-01    16705.0
2011-05-01    16705.0
2011-06-01    16705.0
2011-07-01    16705.0
2011-08-01    16705.0
2011-09-01    16705.0
2011-10-01    16705.0
2011-11-01    16705.0
2011-12-01    16705.0
2012-01-01    16867.0
2012-02-01    16867.0
2012-03-01    16867.0
2012-04-01    16867.0
2012-05-01    16867.0
2012-06-01    16867.0
2012-07-01    168...
2014-06-01    193016.0
2014-07-01    193016.0
2014-08-01    193016.0
2014-09-01    193016.0
2014-10-01    193016.0
2014-11-01    193016.0
2014-12-01    193016.0
2015-01-01    191894.0
2015-02-01    191894.0
2015-03-01    191894.0
2015-04-01    191894.0
2015-05-01    191894.0
2015-06-01    191894.0
2015-07-01    191894.0
2015-08-01    191894.0
2015-09-01    191894.0
2015-10-01    191894.0
2015-11-01    191894.0
2015-12-01    191894.0
Name: 77030, dtype: float64)